<a href="https://colab.research.google.com/github/Collin-Campbell/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [218]:
import numpy as np

In [219]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [220]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 parse_dates=['SALE DATE'],
                 index_col=('SALE DATE'))


In [221]:
# Changing space to underscore in index name

df.index.name = 'SALE_DATE'

In [222]:
# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [223]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [224]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [225]:
print(df.shape)
df.head()

(23040, 20)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0
2019-01-01,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0
2019-01-01,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0
2019-01-01,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0
2019-01-01,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0


In [226]:
# Getting rid of commas from land square ft and converting all values to floats

df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',','')

In [227]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].replace({'': np.NaN, '########': np.NaN})

In [228]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].astype(float)

In [229]:
df['LAND_SQUARE_FEET'].value_counts()

0.0       7500
2000.0    1106
2500.0    1045
4000.0     876
3000.0     369
          ... 
2503.0       1
3578.0       1
4134.0       1
4505.0       1
5204.0       1
Name: LAND_SQUARE_FEET, Length: 3651, dtype: int64

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23040 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNIT

In [231]:
def wrangle(df):
  # Making a copy of the dataset
  df = df.copy()

  # Making a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY 
  # DWELLINGS' and the sale price was more than 100 thousand and less than 2 million
  df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
          (df['SALE_PRICE'] > 100000) & 
          (df['SALE_PRICE'] < 2000000)]

    # Dropping high-cardinality categorical columns
  hc_cols = [col for col in df.select_dtypes('object').columns
             if df[col].nunique() > 11]
  df.drop(columns=hc_cols, inplace=True)

  return df

In [232]:
df = wrangle(df)

In [233]:
df['TAX_CLASS_AT_TIME_OF_SALE'].value_counts()

1    3151
Name: TAX_CLASS_AT_TIME_OF_SALE, dtype: int64

In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 4   BLOCK                           3151 non-null   int64  
 5   LOT                             3151 non-null   int64  
 6   EASE-MENT                       0 non-null      float64
 7   APARTMENT_NUMBER                1 non-null      object 
 8   ZIP_CODE                        3151 non-null   float64
 9   RESIDENTIAL_UNITS               3151 non-null   float64
 10  COMMERCIAL_UNITS                3151 non-null   float64
 11  TOTAL_UNITS                     3151 non-null   float64
 12  LAND_SQUARE_FEET

In [235]:
# Dropping NaN columns, building class column since now they are all the same,
# and tax class at time of sale column since they are also all identical

df = df.drop(['BUILDING_CLASS_CATEGORY', 'EASE-MENT', 'APARTMENT_NUMBER', 'TAX_CLASS_AT_TIME_OF_SALE'], axis=1)

In [236]:
print(df.shape)
df.head()

(3151, 14)


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,
2019-01-01,3,OTHER,1,5495,801,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,A9,550000
2019-01-01,4,OTHER,1,7918,72,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,A1,200000
2019-01-02,2,OTHER,1,4210,19,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,A1,810000
2019-01-02,3,OTHER,1,5212,69,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,A1,125000
2019-01-02,3,OTHER,1,7930,121,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,A5,620000


In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 3   BLOCK                           3151 non-null   int64  
 4   LOT                             3151 non-null   int64  
 5   ZIP_CODE                        3151 non-null   float64
 6   RESIDENTIAL_UNITS               3151 non-null   float64
 7   COMMERCIAL_UNITS                3151 non-null   float64
 8   TOTAL_UNITS                     3151 non-null   float64
 9   LAND_SQUARE_FEET                3151 non-null   float64
 10  GROSS_SQUARE_FEET               3151 non-null   float64
 11  YEAR_BUILT                      3151 non-null   float64
 12  BUILDING_CLASS_A

In [238]:
# Splitting Data

# splitting into target and feature matrix
target = 'SALE_PRICE'
y = df[target]
X = df.drop(columns=target)

In [239]:
# splitting into training and test sets:
# Using data from January — March 2019 to train. Using data from April 2019 to test

cutoff = '2019-04-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [240]:
# Establishing Baseline

y_pred = [y_train.mean()] * len(y_train)

In [241]:
from sklearn.metrics import mean_absolute_error

print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Baseline MAE: 214721.52773001452


In [ ]:
# Applying transformer: OneHotEncoder

# Step 1: Importing the transformer class
from category_encoders import OneHotEncoder, OrdinalEncoder

# Step 2: Instantiating the transformer
ohe = OneHotEncoder(use_cat_names=True)

# Step 3: Fitting my TRAINING data to the transfomer
ohe.fit(X_train)

# Step 4: Transforming
XT_train = ohe.transform(X_train)

In [243]:
print(len(XT_train.columns))
XT_train.columns

33


Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_BEDFORD STUYVESANT', 'NEIGHBORHOOD_FOREST HILLS',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_ASTORIA',
       'TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D', 'BLOCK', 'LOT',
       'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A9',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A5',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A0',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A2',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0'],
      dtype='object')

In [244]:
print(XT_train.shape)
XT_train.head()

(2507, 33)


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
SALE_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1,0,0,0,0,1,0,0,0,0,0,1,0,5495,801,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,0,0,0,0,0,0,0,0,0,0
2019-01-01,0,1,0,0,0,1,0,0,0,0,0,1,0,7918,72,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,0,1,0,0,0,0,0,0,0,0,0
2019-01-02,0,0,1,0,0,1,0,0,0,0,0,1,0,4210,19,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,0,1,0,0,0,0,0,0,0,0,0
2019-01-02,1,0,0,0,0,1,0,0,0,0,0,1,0,5212,69,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,0,1,0,0,0,0,0,0,0,0,0
2019-01-02,1,0,0,0,0,1,0,0,0,0,0,1,0,7930,121,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,0,0,1,0,0,0,0,0,0,0,0


In [248]:
# Performing feature selection with SelectKBest

# Importing the feature selector utility:
from sklearn.feature_selection import SelectKBest, f_regression

# Creating the selector object with the best k=1 features:
selector = SelectKBest(score_func=f_regression, k=1)

# Running the selector on the training data:
XT_train_selected = selector.fit_transform(XT_train, y_train)

# Finding the features that were selected:
selected_mask = selector.get_support()
all_features = XT_train.columns
selected_feature = all_features[selected_mask]

print('The selected feature: ', selected_feature[0])

The selected feature:  GROSS_SQUARE_FEET


In [249]:
# Scaling the ohe data with StandardScaler:

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(XT_train)
XTT_train = ss.transform(XT_train)

In [310]:
# Building Ridge Regression Model:

from sklearn.linear_model import Ridge

model = Ridge(alpha=150)
model.fit(XTT_train, y_train)

Ridge(alpha=150, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [311]:
# Checking metrics:

XT_test = ohe.transform(X_test)
XTT_test = ss.transform(XT_test)

print('RIDGE train MAE', mean_absolute_error(y_train, model.predict(XTT_train)))
print('RIDGE test MAE', mean_absolute_error(y_test, model.predict(XTT_test)))

RIDGE train MAE 151103.0875222934
RIDGE test MAE 155194.34287168915
